In [70]:
from datetime import datetime
from datetime import timezone
import polars as pl
import os
import math
import numpy as np
from sklearn.metrics import r2_score
from plot_data import plot_sensor_measurement, plot_wind_rose, rmse, calc_r2


start_date = datetime(2023, 11, 9, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 11, 10, 23, 59, 59).replace(tzinfo=timezone.utc)

filter = '1h'

sensor_id = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,20]
DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")


# Load Datasets

In [71]:
# Local network measurements dataset
# parquet created by notebook: download_from_hermes.ipynb
# parquet created by notebook: download_from_hermes.ipynb
df = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "download", "acropolis.parquet")) \
.with_columns(pl.col("creation_timestamp").dt.cast_time_unit("us")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date)).collect()
    
# Calibration Correcteed dataset
# parquet created by notebook: processing_pipeline.ipynb
df_cor = pl.scan_parquet(os.path.join(DATA_DIRECTORY,"processed", "1h_cal_corr_acropolis.parquet"))
df_cor = df_cor.sort("creation_timestamp").filter(pl.col("creation_timestamp").is_between(start_date, end_date)).collect()

In [72]:
import plotly.express as px

fig = px.line(df_cor, x="creation_timestamp", y=f"CO2_corr", markers=True, title = "CO2", color="system_name")
fig.show()

fig = px.line(df_cor, x="creation_timestamp", y=f"diff", markers=True, title = "CO2", color="system_name")
fig.show()

fig = px.line(df_cor, x="creation_timestamp", y=f"slope", markers=True, title = "slope", color="system_name")
fig.show()

fig = px.line(df_cor, x="creation_timestamp", y=f"intercept", markers=True, title = "intercept", color="system_name")
fig.show()


In [73]:
# temp_id = 6

# mean_offset = df_corr_agg.filter(pl.col("system_name")==f"tum-esm-midcost-raspi-{temp_id}").select("diff").mean().item()

# l_acropolis = df_corr_agg.filter(pl.col("system_name")==f"tum-esm-midcost-raspi-{temp_id}").select("CO2_corr").with_columns(pl.col("CO2_corr")+ mean_offset).to_series().to_list()

# l_picarro = df_corr_agg.filter(pl.col("system_name")=="Picarro").select("CO2_corr").to_series().to_list()

# print("\nOffset:")
# print(mean_offset)

# print("\nRMSE:")
# print(rmse(l_picarro,l_acropolis))

# print("\nR2:")
# print(calc_r2(l_picarro,l_acropolis))

# Plot other data directly from local database measurements.parquet

In [74]:
plot_sensor_measurement(df,sensor_id=sensor_id,col_name="gmp343_raw",filter=filter)

In [75]:
plot_sensor_measurement(df,sensor_id,col_name="gmp343_compensated",filter=filter)

In [76]:
plot_sensor_measurement(df,sensor_id,col_name="gmp343_filtered",filter=filter)

In [77]:
# plot wind at 3m pole inlet
plot_wind_rose(df,6, 'TUM')
# Maisach
plot_wind_rose(df,1, 'Maisach')
# Rechts-der-Isar
plot_wind_rose(df,2, 'Rechts-der-Isar')
# Finsing
plot_wind_rose(df,3, 'Finsing')
# Großhadern
plot_wind_rose(df,4, 'Großhadern')
# Großhadern
plot_wind_rose(df,5, 'Pasing')
# Feldkirchen
plot_wind_rose(df,7, 'Feldkirchen')
# Taufkirchen
plot_wind_rose(df,8, 'Taufkirchen')

In [78]:
plot_sensor_measurement(df,sensor_id,col_name="wxt532_speed_avg",filter=filter)

In [79]:
plot_sensor_measurement(df,sensor_id,col_name="wxt532_direction_avg",filter=filter)

In [80]:
# Plot auxilliary data
plot_sensor_measurement(df,sensor_id,col_name="sht45_humidity",filter=filter)

In [81]:
plot_sensor_measurement(df,sensor_id,col_name="bme280_humidity",filter=filter)

In [82]:
plot_sensor_measurement(df,sensor_id,col_name="sht45_temperature",filter=filter)

In [83]:
plot_sensor_measurement(df,sensor_id,col_name="bme280_temperature",filter=filter)

In [84]:
plot_sensor_measurement(df,sensor_id,col_name="bme280_pressure",filter=filter)

In [85]:
plot_sensor_measurement(df,sensor_id,col_name="gmp343_temperature",filter=filter)

In [86]:
plot_sensor_measurement(df,sensor_id,col_name="raspi_cpu_usage",filter=filter)

In [87]:
plot_sensor_measurement(df,sensor_id,col_name="raspi_disk_usage",filter=filter)


In [88]:
plot_sensor_measurement(df,sensor_id,col_name="raspi_cpu_temperature",filter=filter)

In [89]:
plot_sensor_measurement(df,sensor_id,col_name="enclosure_bme280_humidity",filter=filter)

In [90]:
plot_sensor_measurement(df,sensor_id,col_name="enclosure_bme280_pressure",filter=filter)

In [91]:
plot_sensor_measurement(df,sensor_id,col_name="enclosure_bme280_temperature",filter=filter)

# Calibration Data

In [92]:
plot_sensor_measurement(df,sensor_id,col_name="cal_bottle_id",filter= '30s')

In [93]:
plot_sensor_measurement(df,sensor_id,col_name="cal_gmp343_filtered",filter= '30s')

In [94]:
plot_sensor_measurement(df,sensor_id,col_name="cal_sht45_humidity",filter= '30s')

In [95]:
plot_sensor_measurement(df,sensor_id,col_name="cal_sht45_temperature",filter= '30s')

In [96]:
plot_sensor_measurement(df,sensor_id,col_name="cal_gmp343_temperature",filter= '30s')

In [97]:
plot_sensor_measurement(df,sensor_id,col_name="cal_bme280_temperature",filter= '30s')

In [98]:
plot_sensor_measurement(df,sensor_id,col_name="cal_bme280_humidity",filter= '30s')

In [99]:
plot_sensor_measurement(df,sensor_id,col_name="cal_bme280_pressure",filter= '30s')

In [100]:
# Plot 400ppm Calibration Bottle 
plot_sensor_measurement(df,sensor_id,col_name="cal_gmp343_filtered",filter= '1m',cut_above= 450,cut_below=350)

In [101]:
# Plot 800ppm Calibration Bottle 
plot_sensor_measurement(df,sensor_id,col_name="cal_gmp343_filtered",filter= '30s',cut_below=700)